In [1]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as Nan
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

## Analysing Diploma Exam Data - results by school

 <img src="images/Alberta_education.jpg" width="200px" align="right"/>
 Provincial diploma exam results are located here: https://education.alberta.ca/diploma-exam-administration/diploma-results/?searchMode=3 .

We will download provincial results by shool xlsx spreadsheet: diploma-multiyear-sch-list-annual.xlsx

In [3]:
school_results = pd.read_excel('https://education.alberta.ca/media/3680580/diploma-multiyear-sch-list-annual.xlsx')
school_results.head()

,Diploma Course,Authority Type,Authority Code,Authority Name,School Code,School Name,2013 Sch Students Writing,2013 Sch School Mark % Exc,2013 Sch School Mark % Acc,2013 Sch School Average %,...,2017 Sch School Mark % Exc,2017 Sch School Mark % Acc,2017 Sch School Average %,2017 Sch School Standard Deviation %,2017 Sch Exam Mark % Exc,2017 Sch Exam Mark Exc Sig,2017 Sch Exam Mark % Acc,2017 Sch Exam Mark Acc Sig,2017 Sch Exam Average %,2017 Sch Exam Standard Deviation %
0,Biology 30,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,100,59.0,96.0,79.5,...,58.8,99.1,79.6,12.8,44.7,=,86.8,=,73.9,16.4
1,Chemistry 30,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,56,26.8,83.9,67.7,...,53.5,98.0,78.0,13.0,55.6,=,93.9,=,77.6,16.7
2,English Lang Arts 30-1,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,111,42.3,99.1,74.6,...,36.1,100.0,74.5,11.0,21.3,=,96.7,=,69.3,11.2
3,English Lang Arts 30-2,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,21,9.5,95.2,69.5,...,18.2,100.0,71.9,7.4,36.4,=,100.0,=,75.8,8.7
4,Mathematics 30-1,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,84,36.9,95.2,73.0,...,50.9,94.7,78.0,15.2,50.9,n/a,87.7,n/a,74.5,20.2


This data set needs to be reshaped - year and all the resluts will be split in separate columns. Let's reshape it and remove rows  having  "n/a" in following columns'Sch Students Writing','Sch Exam Mark % Exc','Sch Exam Mark % Acc'.
Examine the result:

In [4]:
years = []
stats = []
for value in school_results.columns.values[6:]:
    year = value[0:4]
    stat = value[5:]
    int(year)
    if year not in years:
        years.append(year)
    if stat not in stats:
        stats.append(stat)
school_results_reshaped = pd.DataFrame(columns=(list(school_results.columns.values[0:6]) +['Year'] + stats))
for ind,row in school_results.drop(school_results.index[[len(school_results)-2,len(school_results)-1]]).iterrows():
    new_row = pd.DataFrame(columns=(list(school_results.columns.values[0:6]) +['Year'] + stats))
    new_row.loc[0] = [Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan]
    new_row.loc[0][0:6] = row[0:6]                                       
    for year in years:
        new_row.loc[0]['Year'] = year
        for stat in stats:
            new_row.loc[0][stat] = row[year+" "+stat]
        school_results_reshaped=school_results_reshaped.append(new_row)                               
school_results_reshaped = school_results_reshaped.reset_index(drop=True)
school_results_reshaped[['Sch Students Writing','Sch Exam Mark % Exc','Sch Exam Mark % Acc']]=school_results_reshaped[['Sch Students Writing','Sch School Mark % Exc','Sch School Mark % Acc']].apply(pd.to_numeric, errors='coerce')
school_results_reshaped[['Sch Students Writing','Sch Exam Mark % Exc','Sch Exam Mark % Acc']]=school_results_reshaped[['Sch Students Writing','Sch Exam Mark % Exc','Sch Exam Mark % Acc']].dropna()
school_results_reshaped 

,Diploma Course,Authority Type,Authority Code,Authority Name,School Code,School Name,Year,Sch Students Writing,Sch School Mark % Exc,Sch School Mark % Acc,Sch School Average %,Sch School Standard Deviation %,Sch Exam Mark % Exc,Sch Exam Mark Exc Sig,Sch Exam Mark % Acc,Sch Exam Mark Acc Sig,Sch Exam Average %,Sch Exam Standard Deviation %
0,Biology 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2013,100.0,59.0,96.0,79.5,13.5,59.0,=,96.0,=,74.3,14.4
1,Biology 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2014,84.0,56.0,97.6,77.9,14.2,56.0,=,97.6,=,73.9,16.4
2,Biology 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2015,102.0,45.1,98.0,74.2,13.7,45.1,=,98.0,=,72.1,15.7
3,Biology 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2016,104.0,49.0,99.0,77.6,13.0,49.0,=,99.0,=,76.0,14.5
4,Biology 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2017,114.0,58.8,99.1,79.6,12.8,58.8,=,99.1,=,73.9,16.4
5,Chemistry 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2013,56.0,26.8,83.9,67.7,14.7,26.8,=,83.9,=,70.3,18.7
6,Chemistry 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2014,80.0,38.8,97.5,73.9,14.3,38.8,=,97.5,=,74.1,16.7
7,Chemistry 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2015,100.0,27.0,89.0,69.1,15.2,27.0,=,89.0,=,69.0,19.4
8,Chemistry 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2016,104.0,33.7,94.2,72.9,14.9,33.7,=,94.2,=,72.9,18.2
9,Chemistry 30,Charter,9,Foundations for the Future Charter Academy Cha...,12,FFCA High School Campus,2017,99.0,53.5,98.0,78.0,13.0,53.5,=,98.0,=,77.6,16.7


Creating interactive plot - choose school and subject and get 'Sch Students Writing','Sch Exam Mark % Acc' and 'Sch Exam Mark % Exc' for all available years:

In [5]:
from ipywidgets import interact
init_notebook_mode(connected=True)

def view_schools(school,subject):

    result = school_results_reshaped[school_results_reshaped['School Name'] == school]
    result = result[result['Diploma Course'] == subject]

    trace1 = go.Bar( x= result['Year'], y=result['Sch Students Writing'],
                name='Number of students taken the course', marker=dict(color='#59606D'))

    trace2 = go.Bar(x=result['Year'], y=result['Sch Students Writing']/100*result['Sch Exam Mark % Acc'],
                name='Number of students achieved acceptable standard', marker=dict(color='#ffcdd2'))

    trace3 = go.Bar(x=result['Year'], y=result['Sch Students Writing']/100*result['Sch Exam Mark % Exc'],
        name='Number of students achieved standard of excellence',marker=dict(color='#A2D5F2'))


    data = [trace1, trace2, trace3]
    layout = go.Layout(title=subject + "/"+ school,
                xaxis=dict(title='Year'),
                yaxis=dict(title='Number of students'))
    fig = go.Figure(data=data, layout=layout)

    iplot(fig)


interact(view_schools, school=school_results_reshaped['School Name'].unique(), subject=school_results_reshaped['Diploma Course'].unique());

A Jupyter Widget

#### Comparing multiple schools using plotly

In [6]:
def view_schools(school1,school2,school3,school4, school5, subject):

    result1 = school_results_reshaped[school_results_reshaped['School Name'] == school1]
    result1 = result1[result1['Diploma Course'] == subject]
    
    result2 = school_results_reshaped[school_results_reshaped['School Name'] == school2]
    result2 = result2[result2['Diploma Course'] == subject]
    
    result3 = school_results_reshaped[school_results_reshaped['School Name'] == school3]
    result3 = result3[result3['Diploma Course'] == subject]
    
    result4 = school_results_reshaped[school_results_reshaped['School Name'] == school4]
    result4 = result4[result4['Diploma Course'] == subject]
    
    result5 = school_results_reshaped[school_results_reshaped['School Name'] == school5]
    result5 = result5[result5['Diploma Course'] == subject]
    
    trace1 = go.Scatter(
    x = result1['Year'],
    y = result1['Sch Exam Mark % Exc'],
    mode = 'lines+markers',
    name = school1)
        
    trace2 = go.Scatter(
    x = result2['Year'],
    y = result2['Sch Exam Mark % Exc'],
    mode = 'lines+markers',
    name = school2)    
        
    trace3 = go.Scatter(
    x =  result3['Year'],
    y = result3['Sch Exam Mark % Exc'],
    mode = 'lines+markers',
    name = school3)
        
    trace4 = go.Scatter(
    x = result4['Year'],
    y = result4['Sch Exam Mark % Exc'],
    mode = 'lines+markers',
    name = school4)

        
    trace5 = go.Scatter(
    x = result5['Year'],
    y = result5['Sch Exam Mark % Exc'],
    mode = 'lines+markers',
    name = school5)
        
    data = [trace1, trace2, trace3, trace4, trace5]
    layout = go.Layout(title=subject + " - percentage of students achived certificate of excellence",
                xaxis=dict(title='Year'),
                yaxis=dict(title='% of students'))
    fig = go.Figure(data=data, layout=layout)

    iplot(fig)


interact(view_schools, school1=school_results_reshaped['School Name'].unique(),
         school2=school_results_reshaped['School Name'].unique(),
         school3=school_results_reshaped['School Name'].unique(),
         school4=school_results_reshaped['School Name'].unique(),
         school5=school_results_reshaped['School Name'].unique(),
         subject=school_results_reshaped['Diploma Course'].unique());

A Jupyter Widget